# Day 6 - Lab 1: Building RAG Systems

**Objective:** Build a RAG (Retrieval-Augmented Generation) system orchestrated by LangGraph, scaling in complexity from a simple retriever to a multi-agent team that includes a grader and a router.

**Estimated Time:** 180 minutes

**Introduction:**
Welcome to Day 6! Today, we build one of the most powerful and common patterns for enterprise AI: a system that can answer questions about your private documents. We will use LangGraph to create a 'research team' of AI agents. Each agent will have a specific job, and LangGraph will act as the manager, orchestrating their collaboration to find the best possible answer.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We need several libraries for this lab. `langgraph` is the core orchestrator, `langchain` provides the building blocks, `faiss-cpu` is for our vector store, and `pypdf` is for loading documents.

**Model Selection:**
For RAG and agentic workflows, models with strong instruction-following and reasoning are best. `gpt-4.1`, `o3`, or `gemini-2.5-pro` are excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `load_artifact()`: To read the project documents that will form our knowledge base.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langgraph')
install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('faiss-cpu')
install_if_missing('pypdf')

from utils import setup_llm_client, load_artifact
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

faiss-cpu not found, installing...


2025-11-04 13:57:30,169 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: Building the Knowledge Base

An agent is only as smart as the information it can access. We will create a vector store containing all the project artifacts we've created so far. This will be our agent's 'knowledge base'.

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def create_knowledge_base(file_paths):
    """Loads documents from given paths and creates a FAISS vector store.""" 
    all_docs = []
    for path in file_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            docs = loader.load()
            for doc in docs:
                doc.metadata={"source": path} # Add source metadata
            all_docs.extend(docs)
        else:
            print(f"Warning: Artifact not found at {full_path}")

    if not all_docs:
        print("No documents found to create knowledge base.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(all_docs)
    
    print(f"Creating vector store from {len(splits)} document splits...")
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore.as_retriever()

all_artifact_paths = ["artifacts/day1_prd_2025-10-28_14-03-52.md", "artifacts/schema.sql", "artifacts/adr_001_database_choice.md"]
retriever = create_knowledge_base(all_artifact_paths)

Creating vector store from 28 document splits...


## Step 3: The Challenges

### Challenge 1 (Foundational): A Simple RAG Graph

**Task:** Build a simple LangGraph with two nodes: one to retrieve documents and one to generate an answer.

> **Tip:** Think of `AgentState` as the shared 'whiteboard' for your agent team. Every agent (or 'node' in the graph) can read from and write to this state, allowing them to pass information to each other as they work on a problem.

**Instructions:**
1.  Define the state for your graph using a `TypedDict`. It should contain keys for `question` and `documents`.
2.  Create a "Retriever" node. This is a Python function that takes the state, uses the `retriever` to get relevant documents, and updates the state with the results.
3.  Create a "Generator" node. This function takes the state, creates a prompt with the question and retrieved documents, calls the LLM, and stores the answer.
4.  Build the `StateGraph`, add the nodes, and define the edges (`RETRIEVE` -> `GENERATE`).
5.  Compile the graph and invoke it with a question about your project.

**Expected Quality:** A functional graph that can answer a simple question (e.g., "What is the purpose of this project?") by retrieving context from the project artifacts.

In [ ]:
# Single-Agent RAG Graph (Challenge 1)
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph, END, START
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from utils import get_completion

llm = ChatOpenAI(model=model_name)

class AgentState(TypedDict, total=False):
    question: str
    documents: List[Document]
    answer: str
    grade: str # add for future challenge

# --- Node Definitions ---
# Retriever node: pulls relevant documents for the question
def retrieve(state: AgentState) -> AgentState:
    question = state["question"]
    if not question:
        return {}
    if retriever is None:
        print("[Retriever] No retriever available.")
        return {}
    docs = retriever.get_relevant_documents(question)
    print(f"[Retriever] Retrieved {len(docs)} documents.")
    return {"documents": docs, "question": question} # include original question, not strictly required as API maintains state object throughout edge transitions

# Generator node: builds a prompt from question + docs and calls LLM
def generate(state):
    print("[Generator] Generating answer...")
    question = state["question"]
    documents = state["documents"]
    prompt = f"""You are an assistant for question-answering tasks. Use the following retrieved context to answer the question. If you don't know the answer, just say that you don't know.\n\nQuestion: {question}\n\nContext: {documents}\n\nAnswer:"""
    answer = llm.invoke(prompt).content
    return {"answer": answer, "question": question, "documents": documents} # include previous step's states

# --- Build Graph ---
graph = StateGraph(AgentState)
graph.add_node("RETRIEVE", retrieve)
graph.add_node("GENERATE", generate)
graph.add_edge(START, "RETRIEVE")
graph.add_edge("RETRIEVE", "GENERATE")
graph.add_edge("GENERATE", END)
app = graph.compile()

# --- Invoke Graph ---
inputs = {"question": "What is the purpose of this project according to the PRD?"}
result = app.invoke(inputs)
print(f"Final answer: {result['answer']}")

[Retriever] Retrieved 4 documents.
[Generator] Generating answer...
Final answer: The purpose of the project according to the PRD (Product Requirements Document) is to revolutionize the new hire experience from day one by transforming a typically fragmented and overwhelming process into a structured, engaging, and efficient journey. The platform aims to provide personalized learning paths, streamline administrative tasks, and foster quicker team integration for new employees, while offering HR and managers critical tools to track progress and provide timely support. The vision is to empower every new hire to become a productive and engaged contributor faster, significantly reducing ramp-up time and enhancing overall employee retention and satisfaction.


### Challenge 2 (Intermediate): A Graph with a Grader Agent

**Task:** Add a second agent to your graph that acts as a "Grader," deciding if the retrieved documents are relevant enough to answer the question.

> **What is a conditional edge?** It's a decision point. After a node completes its task (like our 'Grader'), the conditional edge runs a function to decide which node to go to next. This allows your agent to change its plan based on new information.

**Instructions:**
1.  Keep your `RETRIEVE` and `GENERATE` nodes from the previous challenge.
2.  Create a new "Grader" node. This function takes the state (question and documents) and calls an LLM with a specific prompt: "Based on the question and the following documents, is the information sufficient to answer the question? Answer with only 'yes' or 'no'."
3.  Add a **conditional edge** to your graph. After the `RETRIEVE` node, the graph should go to the `GRADE` node. After the `GRADE` node, it should check the grader's response. If 'yes', it proceeds to the `GENERATE` node. If 'no', it goes to an `END` node, concluding that it cannot answer the question.

**Expected Quality:** A more robust graph that can gracefully handle cases where its knowledge base doesn't contain the answer, preventing it from hallucinating.

In [ ]:
def grader(state: AgentState) -> AgentState:
    print("[Grader] Grading the answer...")
    question = state["question"]
    documents = state["documents"]
    prompt = f"""You are a grader. Given the question and the provided documents, determine if the documents provide sufficient information and context to answer the question. Respond with 'yes' or 'no'.\n\nQuestion: {question}\n\nContext: {documents}\n\nAnswer:"""
    grade = llm.invoke(prompt).content.strip().lower()
    print(f"[Grader] Grade: {grade}")
    return {"grade": grade}

def decide_to_generate(state):
    if state["grade"].lower() == "yes":
        print("DECISION: Documents are relevant. Proceed to generation.")
        return "GENERATE"
    else:
        return END
        return END
    

workflow_v2 = StateGraph(AgentState)
workflow_v2.add_node("RETRIEVE", retrieve)
workflow_v2.add_node("GRADE", grader)
workflow_v2.add_node("GENERATE", generate)

workflow_v2.set_entry_point("RETRIEVE")
workflow_v2.add_edge("RETRIEVE", "GRADE")
workflow_v2.add_conditional_edges("GRADE", decide_to_generate)
workflow_v2.add_edge("GENERATE", END)

app_v2 = workflow_v2.compile()

print("\n--- Invoking Grader Graph with a relevant question ---")
inputs = {"question": "What database schema will we use?"}
result = app_v2.invoke(inputs)
print(f"Final Answer: {result.get('answer', 'Could not answer question.')}")

print("\n--- Invoking Grader Graph with an irrelevant question ---")
inputs = {"question": "What is the weather in Paris?"}
result = app_v2.invoke(inputs)
print(f"Final Answer: {result.get('answer', 'Could not answer question.')}")


--- Invoking Grader Graph with a relevant question ---
[Retriever] Retrieved 4 documents.
[Grader] Grading the answer...
[Grader] Grade: no.
Final Answer: Could not answer question.

--- Invoking Grader Graph with an irrelevant question ---
[Retriever] Retrieved 4 documents.
[Grader] Grading the answer...
[Grader] Grade: no.
Final Answer: Could not answer question.


### Challenge 3 (Advanced): A Multi-Agent Research Team

**Task:** Build a sophisticated "research team" of specialized agents that includes a router to delegate tasks to the correct specialist.

**Instructions:**
1.  **Specialize your retriever:** Create two separate retrievers. One for the PRD (`prd_retriever`) and one for the technical documents (`tech_retriever` for schema and ADRs).
2.  **Define the Agents:**
    * `ProjectManagerAgent`: This will be the entry point and will act as a router. It uses an LLM to decide whether the user's question is about product requirements or technical details, and routes to the appropriate researcher.
    * `PRDResearcherAgent`: A node that uses the `prd_retriever`.
    * `TechResearcherAgent`: A node that uses the `tech_retriever`.
    * `SynthesizerAgent`: A node that takes the collected documents from either researcher and synthesizes a final answer.
3.  **Build the Graph:** Use conditional edges to orchestrate the flow: The entry point is the `ProjectManager`, which then routes to either the `PRD_RESEARCHER` or `TECH_RESEARCHER`. Both of those nodes should then route to the `SYNTHESIZE` node, which then goes to the `END`.

**Expected Quality:** A highly advanced agentic system that mimics a real-world research workflow, including a router and specialist roles, to improve the accuracy and efficiency of the RAG process.

In [ ]:
prd_retriever = create_knowledge_base(["artifacts/day1_prd_2025-10-28_14-03-52.md"])
tech_retriever = create_knowledge_base(["artifacts/schema.sql", "artifacts/adr_001_database_choice.md"])

class ResearchTeamState(TypedDict):
    question: str
    documents: List[Document]
    answer: str

# 2. Define the agent nodes
def prd_researcher(state):
    print("---NODE: PRD RESEARCHER---")
    documents = prd_retriever.invoke(state["question"])
    return {"documents": documents}

def tech_researcher(state):
    print("---NODE: TECH RESEARCHER---")
    documents = tech_retriever.invoke(state["question"])
    return {"documents": documents}

def synthesize_answer(state):
    print("---NODE: SYNTHESIZE ANSWER---")
    prompt = f"Based on the following documents, create a concise answer to the user's question.\n\nQuestion: {state['question']}\n\nDocuments: {state['documents']}"
    answer = llm.invoke(prompt).content
    return {"answer": answer}

def project_manager_router(state):
    print("---NODE: PROJECT MANAGER (ROUTER)---")
    prompt = f"You are a project manager. Based on the user's question, should you route this to the PRD expert or the Technical expert? Answer with 'PRD_RESEARCHER' or 'TECH_RESEARCHER'.\n\nQuestion: {state['question']}"
    decision = llm.invoke(prompt).content
    print(f"PM Decision: Route to {decision}")
    if 'PRD_RESEARCHER' in decision:
        return "PRD_RESEARCHER"
    else:
        return "TECH_RESEARCHER"

# 3. Build the graph
workflow_v3 = StateGraph(ResearchTeamState)
workflow_v3.add_node("PRD_RESEARCHER", prd_researcher)
workflow_v3.add_node("TECH_RESEARCHER", tech_researcher)
workflow_v3.add_node("SYNTHESIZE", synthesize_answer)

workflow_v3.add_conditional_edges("__start__", project_manager_router)
workflow_v3.add_edge("PRD_RESEARCHER", "SYNTHESIZE")
workflow_v3.add_edge("TECH_RESEARCHER", "SYNTHESIZE")
workflow_v3.add_edge("SYNTHESIZE", END)

app_v3 = workflow_v3.compile()

print("\n--- Invoking Research Team with a PRD question ---")
inputs = {"question": "What are the main user personas for this application?"}
result = app_v3.invoke(inputs)
print(f"Final Answer: {result['answer']}")

print("\n--- Invoking Research Team with a technical question ---")
inputs = {"question": "What columns are in the users table?"}
result = app_v3.invoke(inputs)
print(f"Final Answer: {result['answer']}")


Creating vector store from 14 document splits...
Creating vector store from 14 document splits...

--- Invoking Research Team with a PRD question ---
---NODE: PROJECT MANAGER (ROUTER)---
PM Decision: Route to PRD_RESEARCHER
---NODE: PRD RESEARCHER---
---NODE: SYNTHESIZE ANSWER---
Final Answer: The main user personas for this application are:

1. **The Eager Contributor (New Hire):** A new employee, like Sarah, who is eager to start but feels overwhelmed by the amount of information, struggles to determine what to learn first, and faces difficulties in accessing company policies and finding help.

2. **The Onboarding Orchestrator (HR Onboarding Specialist):** An HR specialist, like Mark, who spends a lot of time manually tracking down new hires for incomplete forms and repetitively answering questions, and lacks tools to efficiently track and support new hires’ progress, leading to compliance risks.

3. **The Team Integrator (Engineering Team Lead):** While specific scenarios for this p

## Lab Conclusion

Incredible work! You have now built a truly sophisticated AI system. You've learned how to create a knowledge base for an agent and how to use LangGraph to orchestrate a team of specialized agents to solve a complex problem. You progressed from a simple RAG chain to a system that includes quality checks (the Grader) and intelligent task delegation (the Router). These are the core patterns for building production-ready RAG applications.

> **Key Takeaway:** LangGraph allows you to define complex, stateful, multi-agent workflows as a graph. Using nodes for agents and conditional edges for decision-making enables the creation of sophisticated systems that can reason, delegate, and collaborate to solve problems more effectively than a single agent could alone.